# **This notebook will be used for loading data and feature engineering**

**Let's first import some relevant packages**

In [0]:
# Import needed packages
import types
import pandas as pd
import numpy as np
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

**Let's now retrieve the dataset which we worked on in the previous notebook**

In [2]:
link = 'https://drive.google.com/file/d/1i02U6vplm6aXWbQLNcZwGzt6oBEAoVpP/view?usp=sharing'
fluff, id  = link.split('d/')
id, fluff = id.split('/view')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('abcde.csv')  
df = pd.read_csv('abcde.csv')
df.head()
print(df.shape)

1i02U6vplm6aXWbQLNcZwGzt6oBEAoVpP
(1191331, 24)


In [3]:
# Mount google drive for upload later
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


**Let's select the relevant columns and omit columns which are either duplicates of others, or which won't be needed any further. We will omit day of month, as we would like to generalise the model for other months than January. Same for years**

In [4]:
df = df[["DAY_OF_WEEK","DEP_TIME","DISTANCE","OP_UNIQUE_CARRIER","ORIGIN_AIRPORT_SEQ_ID","DEST_AIRPORT_SEQ_ID","CANCELLED","DIVERTED","ARR_DEL15","DEP_DEL15"]]
df.shape

(1191331, 10)

**Let's remove cancelled flights**

In [5]:
df = df[df["CANCELLED"]==0]
df.shape

(1167677, 10)

**Let's remove diverted flights**

In [6]:
df = df[df["DIVERTED"]==0]
df.shape

(1165231, 10)

**Let's remove the cancelled and diverted columns so we are just left with features and the target**

In [7]:
# take important columns
df = df.drop(columns={"DIVERTED","CANCELLED"})
df.shape

(1165231, 8)

**Let's drop rows with errors/missing values**

In [8]:
df = df.dropna()
df.shape

(1165231, 8)

In [9]:
df

,DAY_OF_WEEK,DEP_TIME,DISTANCE,OP_UNIQUE_CARRIER,ORIGIN_AIRPORT_SEQ_ID,DEST_AIRPORT_SEQ_ID,ARR_DEL15,DEP_DEL15
0,2,601.0,300,9E,1195302,1039707,0.0,0.0
1,2,1359.0,596,9E,1348702,1119302,0.0,0.0
2,2,1215.0,229,9E,1143302,1119302,0.0,0.0
3,2,1521.0,223,9E,1524906,1039707,0.0,0.0
4,2,1847.0,579,9E,1039707,1177801,0.0,0.0
...,...,...,...,...,...,...,...,...
1191326,5,1002.0,443,9E,1541205,1143302,1.0,1.0
1191327,5,1747.0,509,9E,1143302,1247805,0.0,0.0
1191328,5,554.0,610,9E,1199603,1295304,0.0,0.0
1191329,5,1714.0,589,9E,1039707,1591904,0.0,0.0


**Turns out there are none, after removing the cancelled and diverted flights**







**Now let's remove the "gaps" in the departure time column by making minutes out of 100 rather than 60 and then rejoining to make a uniform time scale between 0 and 2399**

In [0]:
# Convert scheduled time into a uniform time scale between 0-2400
df["DEP_HOUR"]=np.floor(df["DEP_TIME"]/100)
df["DEP_MINUTE100"]=(df["DEP_TIME"]-df["DEP_HOUR"]*100)*100/60
df["DEP_TIME_UNIFORM"]=df["DEP_HOUR"]*100+df["DEP_MINUTE100"]

# Remove unneeded columns
df = df.drop(columns = {"DEP_HOUR","DEP_MINUTE100","DEP_TIME"})

df.head()

,DAY_OF_WEEK,DISTANCE,OP_UNIQUE_CARRIER,ORIGIN_AIRPORT_SEQ_ID,DEST_AIRPORT_SEQ_ID,ARR_DEL15,DEP_DEL15,DEP_TIME_UNIFORM
0,2,300,9E,1195302,1039707,0.0,0.0,601.666667
1,2,596,9E,1348702,1119302,0.0,0.0,1398.333333
2,2,229,9E,1143302,1119302,0.0,0.0,1225.000000
3,2,223,9E,1524906,1039707,0.0,0.0,1535.000000
4,2,579,9E,1039707,1177801,0.0,0.0,1878.333333


**Now we will convert cyclical categories into both sine and cosine waves to preserve the cyclical nature of these time data points and ensure no two times of day or days of week are treated the same by the model. These will both be fed into the model**

In [0]:
# Time of day 24 hour cycle
df[["DEP_TIME_UNIFORM_SINE"]]=np.sin(df[["DEP_TIME_UNIFORM"]]/2400*2*np.pi)
df[["DEP_TIME_UNIFORM_COSINE"]]=np.cos(df[["DEP_TIME_UNIFORM"]]/2400*2*np.pi)

# Day of week 7 day cycle 
df[["DAY_OF_WEEK_SINE"]]=np.sin(df[["DAY_OF_WEEK"]]/7*2*np.pi)
df[["DAY_OF_WEEK_COSINE"]]=np.cos(df[["DAY_OF_WEEK"]]/7*2*np.pi)

# Drop original non-cyclical columns
df = df.drop(columns = {"DEP_TIME_UNIFORM","DAY_OF_WEEK"})

df.head()

,DISTANCE,OP_UNIQUE_CARRIER,ORIGIN_AIRPORT_SEQ_ID,DEST_AIRPORT_SEQ_ID,ARR_DEL15,DEP_DEL15,DEP_TIME_UNIFORM_SINE,DEP_TIME_UNIFORM_COSINE,DAY_OF_WEEK_SINE,DAY_OF_WEEK_COSINE
0,300,9E,1195302,1039707,0.0,0.0,0.999990,-0.004363,0.974928,-0.222521
1,596,9E,1348702,1119302,0.0,0.0,-0.496217,-0.868199,0.974928,-0.222521
2,229,9E,1143302,1119302,0.0,0.0,-0.065403,-0.997859,0.974928,-0.222521
3,223,9E,1524906,1039707,0.0,0.0,-0.768842,-0.639439,0.974928,-0.222521
4,579,9E,1039707,1177801,0.0,0.0,-0.979045,0.203642,0.974928,-0.222521


**We will use MinMaxScaler to normalise all numerical data between zero and one**

In [0]:
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

# Day of week 
scaler_dayS = preprocessing.MinMaxScaler()
scaler_dayC = preprocessing.MinMaxScaler()    
df["DAY_OF_WEEK_SINE_MM"] = pd.DataFrame(scaler_dayS.fit_transform(pd.DataFrame(df["DAY_OF_WEEK_SINE"])),columns=["DAY_OF_WEEK_SINE"])
df["DAY_OF_WEEK_COSINE_MM"] = pd.DataFrame(scaler_dayC.fit_transform(pd.DataFrame(df["DAY_OF_WEEK_COSINE"])),columns=["DAY_OF_WEEK_COSINE"])

# Time of day
scaler_timeS = preprocessing.MinMaxScaler() 
scaler_timeC = preprocessing.MinMaxScaler()   
df["DEP_TIME_UNIFORM_SINE_MM"] = pd.DataFrame(scaler_timeS.fit_transform(pd.DataFrame(df["DEP_TIME_UNIFORM_SINE"])),columns=["DEP_TIME_UNIFORM_SINE"])
df["DEP_TIME_UNIFORM_COSINE_MM"] = pd.DataFrame(scaler_timeC.fit_transform(pd.DataFrame(df["DEP_TIME_UNIFORM_COSINE"])),columns=["DEP_TIME_UNIFORM_COSINE"])

# Flight Distance 
scaler_dist = preprocessing.MinMaxScaler() 
df["DISTANCE_MM"] = pd.DataFrame(scaler_dist.fit_transform(pd.DataFrame(df["DISTANCE"])),columns=["DISTANCE"])

# Drop unscaled columns
df = df.drop(columns = {"DEP_TIME_UNIFORM_SINE","DEP_TIME_UNIFORM_COSINE","DAY_OF_WEEK_SINE","DAY_OF_WEEK_COSINE","DISTANCE"})

print(df.shape)
df.head()

(1165231, 10)


,OP_UNIQUE_CARRIER,ORIGIN_AIRPORT_SEQ_ID,DEST_AIRPORT_SEQ_ID,ARR_DEL15,DEP_DEL15,DAY_OF_WEEK_SINE_MM,DAY_OF_WEEK_COSINE_MM,DEP_TIME_UNIFORM_SINE_MM,DEP_TIME_UNIFORM_COSINE_MM,DISTANCE_MM
0,9E,1195302,1039707,0.0,0.0,1.0,0.356896,0.999995,0.497818,0.053120
1,9E,1348702,1119302,0.0,0.0,1.0,0.356896,0.251892,0.065901,0.111572
2,9E,1143302,1119302,0.0,0.0,1.0,0.356896,0.467298,0.001071,0.039100
3,9E,1524906,1039707,0.0,0.0,1.0,0.356896,0.115579,0.180280,0.037915
4,9E,1039707,1177801,0.0,0.0,1.0,0.356896,0.010477,0.601821,0.108215


**We will now one-hot encode the categorical data**

In [0]:
df = pd.get_dummies(df, columns=["OP_UNIQUE_CARRIER","ORIGIN_AIRPORT_SEQ_ID","DEST_AIRPORT_SEQ_ID"])
df.head()

,ARR_DEL15,DEP_DEL15,DAY_OF_WEEK_SINE_MM,DAY_OF_WEEK_COSINE_MM,DEP_TIME_UNIFORM_SINE_MM,DEP_TIME_UNIFORM_COSINE_MM,DISTANCE_MM,OP_UNIQUE_CARRIER_9E,OP_UNIQUE_CARRIER_AA,OP_UNIQUE_CARRIER_AS,OP_UNIQUE_CARRIER_B6,OP_UNIQUE_CARRIER_DL,OP_UNIQUE_CARRIER_EV,OP_UNIQUE_CARRIER_F9,OP_UNIQUE_CARRIER_G4,OP_UNIQUE_CARRIER_HA,OP_UNIQUE_CARRIER_MQ,OP_UNIQUE_CARRIER_NK,OP_UNIQUE_CARRIER_OH,OP_UNIQUE_CARRIER_OO,OP_UNIQUE_CARRIER_UA,OP_UNIQUE_CARRIER_WN,OP_UNIQUE_CARRIER_YV,OP_UNIQUE_CARRIER_YX,ORIGIN_AIRPORT_SEQ_ID_1013505,ORIGIN_AIRPORT_SEQ_ID_1013506,ORIGIN_AIRPORT_SEQ_ID_1013603,ORIGIN_AIRPORT_SEQ_ID_1014005,ORIGIN_AIRPORT_SEQ_ID_1014106,ORIGIN_AIRPORT_SEQ_ID_1014602,ORIGIN_AIRPORT_SEQ_ID_1015502,ORIGIN_AIRPORT_SEQ_ID_1015706,ORIGIN_AIRPORT_SEQ_ID_1015804,ORIGIN_AIRPORT_SEQ_ID_1016506,ORIGIN_AIRPORT_SEQ_ID_1017004,ORIGIN_AIRPORT_SEQ_ID_1018502,ORIGIN_AIRPORT_SEQ_ID_1020803,ORIGIN_AIRPORT_SEQ_ID_1025702,ORIGIN_AIRPORT_SEQ_ID_1026802,ORIGIN_AIRPORT_SEQ_ID_1027903,...,DEST_AIRPORT_SEQ_ID_1489302,DEST_AIRPORT_SEQ_ID_1490505,DEST_AIRPORT_SEQ_ID_1490803,DEST_AIRPORT_SEQ_ID_1495203,DEST_AIRPORT_SEQ_ID_1495503,DEST_AIRPORT_SEQ_ID_1496002,DEST_AIRPORT_SEQ_ID_1498603,DEST_AIRPORT_SEQ_ID_1500803,DEST_AIRPORT_SEQ_ID_1501606,DEST_AIRPORT_SEQ_ID_1502305,DEST_AIRPORT_SEQ_ID_1502403,DEST_AIRPORT_SEQ_ID_1502704,DEST_AIRPORT_SEQ_ID_1504102,DEST_AIRPORT_SEQ_ID_1504803,DEST_AIRPORT_SEQ_ID_1504804,DEST_AIRPORT_SEQ_ID_1507002,DEST_AIRPORT_SEQ_ID_1507003,DEST_AIRPORT_SEQ_ID_1507402,DEST_AIRPORT_SEQ_ID_1509602,DEST_AIRPORT_SEQ_ID_1524906,DEST_AIRPORT_SEQ_ID_1529502,DEST_AIRPORT_SEQ_ID_1530402,DEST_AIRPORT_SEQ_ID_1532305,DEST_AIRPORT_SEQ_ID_1535602,DEST_AIRPORT_SEQ_ID_1537002,DEST_AIRPORT_SEQ_ID_1537604,DEST_AIRPORT_SEQ_ID_1538005,DEST_AIRPORT_SEQ_ID_1538902,DEST_AIRPORT_SEQ_ID_1540103,DEST_AIRPORT_SEQ_ID_1541106,DEST_AIRPORT_SEQ_ID_1541205,DEST_AIRPORT_SEQ_ID_1545405,DEST_AIRPORT_SEQ_ID_1558203,DEST_AIRPORT_SEQ_ID_1560702,DEST_AIRPORT_SEQ_ID_1562404,DEST_AIRPORT_SEQ_ID_1584102,DEST_AIRPORT_SEQ_ID_1591904,DEST_AIRPORT_SEQ_ID_1599102,DEST_AIRPORT_SEQ_ID_1621802,DEST_AIRPORT_SEQ_ID_1686901
0,0.0,0.0,1.0,0.356896,0.999995,0.497818,0.053120,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,1.0,0.356896,0.251892,0.065901,0.111572,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,1.0,0.356896,0.467298,0.001071,0.039100,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,1.0,0.356896,0.115579,0.180280,0.037915,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,1.0,0.356896,0.010477,0.601821,0.108215,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


**We will now drop any rows with errors**

In [0]:
# drop N/A rows

df = df.dropna()
df.shape

(1139267, 770)

**We will set the class labels into a new dataframe y**

In [0]:
# define class
y=df[["ARR_DEL15"]]
y.shape

(1139267, 1)

**We will set the features into a new dataframe x**

In [0]:
# define features

x = df.drop(columns={"ARR_DEL15"})
x.shape

(1139267, 769)

**Upload the features and class labels to google drive for the model development**

In [0]:
x.to_csv('/drive/My Drive/Colab Notebooks/Advanced Data Science/Binary Classifier (v1)/x2---post feature_eng')
y.to_csv('/drive/My Drive/Colab Notebooks/Advanced Data Science/Binary Classifier (v1)/y2---post feature_eng')

print("done")

done


**We will define a function which will upload the pre-fit scalers to google drive for reuse in the deployment phase. This code will overwrite any file with the specified filename if one already exists**

In [0]:
from sklearn.externals import joblib

def uploadscaler():
  global filename

  drive = GoogleDrive(gauth)

  file_list = drive.ListFile({'q':"'1EeZCwYgPr_nntYAYrNNhOrA7Hf0PRJ2p'  in parents and trashed=False"}).GetList()
  
  filefound = 0

  for x in range(len(file_list)):
    if file_list[x]['title'] == filename:
      ident = file_list[x]['id']
      filefound = 1

  if filefound==1:
    file1 = drive.CreateFile({'id':ident,'title' : filename, 'parents': [{'id': '1EeZCwYgPr_nntYAYrNNhOrA7Hf0PRJ2p'}]})
  else:
    file1 = drive.CreateFile({'title' : filename, 'parents': [{'id': '1EeZCwYgPr_nntYAYrNNhOrA7Hf0PRJ2p'}]})
  file1.SetContentFile(filename)
  file1.Upload()

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


**Upload the scalers to google drive**

In [0]:
filename = "scaler_dayS.pkl"
joblib.dump(scaler_dayS, filename)
uploadscaler()

filename = "scaler_dayC.pkl"
joblib.dump(scaler_dayC, filename)
uploadscaler()

filename = "scaler_timeS.pkl"
joblib.dump(scaler_timeS, filename)
uploadscaler()

filename = "scaler_timeC.pkl"
joblib.dump(scaler_timeC, filename)
uploadscaler()

filename = "scaler_dist.pkl"
joblib.dump(scaler_dist, filename)
uploadscaler()

print("done")

done
